In [12]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil import parser

RAW_DATA_PATH = r"../data/raw"
PROCESSED_DATA_PATH = r"../data/processed"

In [2]:
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("Set2")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

pd.set_option("display.max_columns", None)

In [3]:
covid_data = pd.read_csv(os.path.join(RAW_DATA_PATH, "covid_19_data.csv"))

Kiểm tra lại bảng dữ liệu

In [ ]:
covid_data.head(5)

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,Mainland China,2020-01-22 17:00:00,14.0,0.0,0.0
2,3,2020-01-22,Chongqing,Mainland China,2020-01-22 17:00:00,6.0,0.0,0.0
3,4,2020-01-22,Fujian,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
4,5,2020-01-22,Gansu,Mainland China,2020-01-22 17:00:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
95,96,2020-01-24,Shanghai,Mainland China,2020-01-24 17:00:00,20.0,0.0,1.0
96,97,2020-01-24,Jiangxi,Mainland China,2020-01-24 17:00:00,18.0,0.0,0.0
97,98,2020-01-24,Sichuan,Mainland China,2020-01-24 17:00:00,15.0,0.0,0.0
98,99,2020-01-24,Shandong,Mainland China,2020-01-24 17:00:00,15.0,0.0,0.0


***Làm sạch cột ObservationDate và Last Update.***  

Vấn đề: Dữ liệu trong cột ObservationDate và Last Update là kiểu string, các giá trị không có sự thống nhất về định dạng để chuyển đổi sang datetime bằng phương pháp thủ công.  
Giải quyết: Dùng hàm parser.parse() từ thư viện dateutil.  
- Hàm parse() tự động chuyển đổi các giá trị có kiểu string về datetime nếu string đủ thông tin về thời gian.  
- Các giá trị không thể chuyển đổi sẽ gán dưới dạng pd.NaT (Not A Time) để kiểm tra.  

In [19]:
#Hàm chuyển đổi string về datetime
def safe_parse(date_str): 
    try:
        # Sử dụng parser của dateutil để chuyển đổi
        return parser.parse(date_str, dayfirst=True)  # dayfirst=True ưu tiên định dạng DD/MM/YYYY
    except (ValueError, TypeError):
        # Nếu không chuyển đổi được, trả về NaT (Not a Time)
        return pd.NaT

In [ ]:
#Chuyển đổi 2 cột ObservationDate và Last Update về datetime
covid_data['ObservationDate'] = covid_data['ObservationDate'].apply(safe_parse)
covid_data['Last Update'] = covid_data['Last Update'].apply(safe_parse)

In [ ]:
#Kiểm tra có giá trị nào không chuyển đổi thành công hay không
print(covid_data['ObservationDate'].isna().any())
print(covid_data['Last Update'].isna().any())

False
False


Kết quả: Không có dữ liệu lỗi, tất cả đã được chuyển thành công